In [1]:
import pandas as pd
# -*- coding: utf-8 -*-

In [2]:
df = pd.read_fwf('zaliznyak.txt')

In [3]:
for i in range(len(df['0'])):
    df.loc[i, '0'] = df.loc[i, '0'][df.loc[i, '0'].find('#') + 1:]
df.loc[:, '0'] = df.loc[:, '0'].str.split(',')    

In [4]:
df

,0
0,[а]
1,"[аба', абы', абы', а'б, абе', аба'м, абу', абы..."
2,"[абажу'р, абажу'ры, абажу'ра, абажу'ров, абажу..."
3,"[абажу'рный, абажу'рная, абажу'рное, абажу'рны..."
4,"[аба'з, аба'зы, аба'за, аба'зов, аба'зу, аба'з..."
...,...
86834,"[я'щичник, я'щичники, я'щичника, я'щичников, я..."
86835,"[я'щичный, я'щичная, я'щичное, я'щичные, я'щич..."
86836,"[я'щур, я'щуры, я'щура, я'щуров, я'щуру, я'щур..."
86837,"[я'щурка, я'щурки, я'щурки, я'щурок, я'щурке, ..."


In [5]:
accent1 = "'"
accent2 = '`'

In [6]:
d = dict()

In [7]:
VOWELS = ['а', 'у', 'о', 'ы', 'и', 'э', 'я', 'ю', 'ё', 'е']
def get_accent(body):
    body = body.encode(encoding='utf-8')
    responce = requests.post('https://ws3.morpher.ru/russian/addstressmarks', headers={'Content-type': 'text/plain; charset=utf-8'},  data=(body))
    s = responce.text
    s = s[46:-9]
    print(s)
    return s
def get_cnt_vowels(s):
    cnt_vowels = 0
    for vowel in VOWELS:
        cnt_vowels += s.count(vowel)
    return cnt_vowels    
accent = u'\u0301'
def get_id_accent(s):
    id_accent = -1
    if accent in s:
        id_accent = s.find(accent) - 1
    elif 'ё' in s:
        id_accent = s.find('ё')
    elif get_cnt_vowels(s) == 1:
        for vowel in VOWELS:
            if vowel in s:
                id_accent = s.find(vowel)
    return id_accent

In [11]:
d.clear()
for i in range(len(df['0'])):
    for j in range(len(df.loc[i, '0'])):
        id_accent = df.loc[i, '0'][j].find(accent1) - 1
        cnt_vowels = 0
        word = df.loc[i, '0'][j].replace(accent1, '').replace(accent2, '') 
        if id_accent == -2:
            id_accent = get_id_accent(word)    
        for k in range(len(df.loc[i, '0'][j])):
            if df.loc[i, '0'][j][k] in VOWELS:
                cnt_vowels += 1
            if k == id_accent:
                id_accent = cnt_vowels - 1
                break         
        d[word] = id_accent

In [12]:
d

{'а': 0,
 'аба': 1,
 'абы': -1,
 'аб': 0,
 'абе': 1,
 'абам': 1,
 'абу': 1,
 'абой': 1,
 'абою': 1,
 'абами': 1,
 'абах': 1,
 'абажур': 2,
 'абажуры': 2,
 'абажура': 2,
 'абажуров': 2,
 'абажуру': 2,
 'абажурам': 2,
 'абажуром': 2,
 'абажурами': 2,
 'абажуре': 2,
 'абажурах': 2,
 'абажурный': 2,
 'абажурная': 2,
 'абажурное': 2,
 'абажурные': 2,
 'абажурного': 2,
 'абажурной': 2,
 'абажурных': 2,
 'абажурному': 2,
 'абажурным': 2,
 'абажурную': 2,
 'абажурною': 2,
 'абажурными': 2,
 'абажурном': 2,
 'абажурен': 2,
 'абажурна': 2,
 'абажурно': 2,
 'абажурны': 2,
 'абажурнее': 2,
 'поабажурнее': 3,
 'абажурней': 2,
 'поабажурней': 3,
 'абаз': 1,
 'абазы': 1,
 'абаза': 1,
 'абазов': 1,
 'абазу': 1,
 'абазам': 1,
 'абазом': 1,
 'абазами': 1,
 'абазе': 1,
 'абазах': 1,
 'абазин': 2,
 'абазины': 2,
 'абазина': 2,
 'абазину': 2,
 'абазинам': 2,
 'абазином': 2,
 'абазинами': 2,
 'абазине': 2,
 'абазинах': 2,
 'абазинец': 2,
 'абазинцы': 2,
 'абазинца': 2,
 'абазинцев': 2,
 'абазинцу': 2,
 'аба

In [53]:
def metr_classifier(verse):
    verse_list = [i.split() for i in verse]
    score = 0
    add_score = [[0.5, -1], [-0.25, 1.5]]
    all_cnt_vowels = 0
    for line in verse_list:
        cnt_vowels = 0
        d_list = []
        for word in line:
            if d.get(word, -1) == -1:
                cnt_vowels += get_cnt_vowels(word)
                continue
            if d[word] != -1:
                d_list.append(cnt_vowels + d[word])
            cnt_vowels += get_cnt_vowels(word)  
        for i_vowel in range(cnt_vowels):
            score += add_score[i_vowel % 2][int(i_vowel in d_list)]
        all_cnt_vowels += cnt_vowels
    return score / all_cnt_vowels

In [55]:
s =  """рыба старую песню мне сердце поёт
        старые
        скаты
        иду я к тебе в тумане тепловозы
        о дне томящем не жалей
        всех времен
        цари тираны
        в жизни забвения
        как с ней отлично был знаком
        когда советский трудовой народ
        вот дух философских начал
        давно отплакали метели
        лихое время нищеты
        нищеты и тщеты и заботы
        о тебе не попадался""".lower().split('\n')
s1 = '''бабушка ты тоже
маленькой была
как будто беседуют с ней немного
что в тебя я тоскую безумно
ты любила меня
для тебя и на землю падает целует ноги
так вот что мне в даровании поэта
если
вот смысл глубочайший искусства
вот и живу  необрученным
и я выходил без пальто
сегодня пришла и устроили нам
любая беда
утешаться верить
ворожить тужить
плакать лицемерить
стоит за ней
свои слезы как прячут березы
горький сок под корою в морозы
        '''.lower().split('\n')
t = '''Скажи ка дядя ведь не даром
Москва спаленная пожаром
Французу отдана
Ведь были ж схватки боевые
Да говорят еще какие
Недаром помнит вся Россия
Про день Бородина
        '''.lower().split('\n')
t1 = '''В России ямбами поэты
Предпочитают сочинять
В стопе такой одна примета
Где чётный слог там ударять
Поможет выявить нам ямб
Повтор ВодА водА водА
ВодА водА водА водА
Звучит так ямба ритм всегда'''.lower().split('\n')
metr_classifier(s), metr_classifier(s1), metr_classifier(t), metr_classifier(t1)

(0.23214285714285715, 0.15, 0.4473684210526316, 0.6515151515151515)